# **Car Accident Severity in Seattle**
## Applied Data Science Capstone - Coursera

NOTE: This notebook shows the process of building a machine learning model for accident severity prediction. It is part of the final capstone project in Coursera to obtain the IBM Professional Certificate in Data Science.

## **Introduction**

Car accidents happen every day for a variety of reasons and these have significant socioeconomic costs. Efforts to raise drivers' awareness towards mindful driving have been promoted across the USA and the authorities try to provide the conditions (e.g. road signs, traffic lights, traffic information, radars) to mitigate the probability of accidents happening. 
Today we have the data and the modeling capacities to even better understand the conditions that promote severe accidents and this project intends to build a machine learning model to better inform decision-makers in the city of Seattle using available data. This model will help the authorities to take appropriate measures to reduce accident severity and improve traffic safety.


## **Data**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df= pd.read_csv(r'C:\Users\marco\Desktop\Data Science\IBM Coursera\Capstone project\Data-Collisions.csv')

The data was provided by the Seattle Police Department and corresponds to collisions registered between 2004 and 2020. The data is stored in a CSV file, presenting 38 columns and 194673 rows. It describes the details of each accident, including weather conditions, collision type, date/time of accident and location.

In the dataset we have 3 types of variables: integers (12), floats (4) and objects (22), as we can see below:  

In [ ]:
df.dtypes

The variable SEVERITYCODE encodes the Seattle Department of Transport accident severity metric and this will be our 'dependent variable' (the variable we want to predict). The numerical codes and their meaning are as follows:

* 0: Unknown
* 1: Property damage
* 2: Injury
* 2b: Serious injury
* 3: Fatality

In [ ]:
df['SEVERITYCODE'].value_counts().to_frame()

By analysing the dataset, we can see that there are only two levels (out of five) of 'severity' registered:
- 1: 136485 registrations
- 2: 58188 registrations

The data is unbalanced, since we have many more instances of 'severity 1' compared with 'severity 2'. Data must be balanced and normalized in the data processsing step.

We have 37 attributes (columns) that can be used for building the model , but not all are useful.

At this stage, the following columns were dropped from the dataset as they were deemed not useful for the model (e.g. unnecessary, uninformative or redundant columns).

In [ ]:
df.drop(columns=['JUNCTIONTYPE','INCDATE','PEDROWNOTGRNT','ST_COLCODE','PEDCYLCOUNT','PERSONCOUNT','PEDCOUNT','COLLISIONTYPE', 'OBJECTID', 'X', 'Y', 'INCKEY', 'REPORTNO', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'STATUS', 'COLDETKEY', 'LOCATION', 'INTKEY', 'INCDTTM','SDOT_COLDESC', 'SDOT_COLCODE', 'INATTENTIONIND', 'SDOTCOLNUM', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY'], inplace= True)
df.head()

## **Methodology**

Now we will perform a data wrangling step to prepare the dataset for analysis. First lets look at missing data:

In [ ]:
#Looking for missing data
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("---------------------") 

From the output above, we can see that columns 'SPEEDING' has much more missing data cells than not. Therefore we will drop this columns from our analysis. Columns 'ADDRTYPE', 'JUNCTIONTYPE', 'UNDERINFL', 'WEATHER', 'ROADCOND' and 'LIGHTCOND' have some missing data. In this case, we will drop the rows with missing values on those columns.

In [ ]:
df.drop(columns=['SPEEDING'], inplace= True)

In [ ]:
df=df.dropna(axis=0)
df.head()

In the column 'UNDERINFL' we have a mix of numerical (0 and 1) and categorical data (Y and N). We will convert Y to 1 and N to 0 to uniformize the data to numerical.

In [ ]:
df['UNDERINFL'].replace('Y', 1, inplace=True)
df['UNDERINFL'].replace('N', 0, inplace=True)
df['UNDERINFL']= df['UNDERINFL'].astype('int')

Lets also convert 'HITPARKEDCAR' to numeric variables 0 and 1

In [ ]:
df['HITPARKEDCAR'].replace('Y', 1, inplace=True)
df['HITPARKEDCAR'].replace('N', 0, inplace=True)
df['HITPARKEDCAR']= df['HITPARKEDCAR'].astype('int')

Moreover, in column 'LIGHTCOND' we will merge the categorie levels 'Dark - No Street Lights' and 'Dark - Street Lights Off' to a single category 'Dark - Street Light Off'. We will eliminate rows with 'Other' and 'Dark - Unknown Lighting' as they represent a small fraction of cases and do not provide relevant information. Let first look at value counts.

In [ ]:
df['LIGHTCOND'].value_counts().to_frame()

In [ ]:
df['LIGHTCOND'].replace('Dark - No Street Lights', 'Night', inplace=True)
df['LIGHTCOND'].replace('Dusk', 'Dusk/Dawn', inplace=True)
df['LIGHTCOND'].replace('Dawn', 'Dusk/Dawn', inplace=True)
df['LIGHTCOND'].replace('Dark - Street Lights On', 'Night', inplace=True)
df['LIGHTCOND'].replace('Dark - Street Lights Off', 'Night', inplace=True)
indexNames = df[df['LIGHTCOND'] == 'Other'].index
df.drop(indexNames, inplace=True)

In [ ]:
indexNames = df[df['LIGHTCOND'] == 'Dark - Unknown Lighting'].index
df.drop(indexNames, inplace=True)

In [ ]:
indexNames = df[df['LIGHTCOND'] == 'Unknown'].index
df.drop(indexNames, inplace=True)

Columns 'WEATHER' will also have merged and dropped categories to reduce category levels. New category 'Elements' will include all categories that involve weather elements like rain, snow and wind. We will drop 'Partly Cloudy', 'Unknown' and 'Other'

In [ ]:
df['WEATHER'].value_counts().to_frame()

In [ ]:
df['WEATHER'].replace('Raining', 'Elements', inplace=True)
df['WEATHER'].replace('Snowing', 'Elements', inplace=True)
df['WEATHER'].replace('Sleet/Hail/Freezing Rain', 'Elements', inplace=True)
df['WEATHER'].replace('Raining', 'Elements', inplace=True)
df['WEATHER'].replace('Fog/Smog/Smoke', 'Elements', inplace=True)
df['WEATHER'].replace('Blowing Sand/Dirt', 'Elements', inplace=True)
df['WEATHER'].replace('Severe Crosswind', 'Elements', inplace=True)

In [ ]:
indexNames = df[df['WEATHER'] == 'Other'].index
df.drop(indexNames, inplace=True)

In [ ]:
indexNames = df[df['WEATHER'] == 'Unknown'].index
df.drop(indexNames, inplace=True)

In [ ]:
indexNames = df[df['WEATHER'] == 'Partly Cloudy'].index
df.drop(indexNames, inplace=True)

Columns 'ROADCOND' will also have merged and dropped categories to reduce category levels. New category 'Elements' will include all categories that involve elements like water, sand and ice. We will drop 'Unknown' and 'Other'

In [ ]:
df['ROADCOND'].value_counts().to_frame()

In [ ]:
df['ROADCOND'].replace('Ice', 'Elements', inplace=True)
df['ROADCOND'].replace('Wet', 'Elements', inplace=True)
df['ROADCOND'].replace('Snow/Slush', 'Elements', inplace=True)
df['ROADCOND'].replace('Standing Water', 'Elements', inplace=True)
df['ROADCOND'].replace('Sand/Mud/Dirt', 'Elements', inplace=True)
df['ROADCOND'].replace('Oil', 'Elements', inplace=True)

In [ ]:
indexNames = df[df['ROADCOND'] == 'Other'].index
df.drop(indexNames, inplace=True)

In [ ]:
indexNames = df[df['ROADCOND'] == 'Unknown'].index
df.drop(indexNames, inplace=True)

In [ ]:
df.head()

As we saw above, 'SEVERITYCODE' (the dependent varaiable we want to predict based on selected features) is unbalanced, since we have many more cells with severity code '1' than '2'. Let's use resampling to balance the data.

In [ ]:
df['SEVERITYCODE'].value_counts().to_frame()

In [ ]:
from sklearn.utils import resample
df_sevcode_1= df[df["SEVERITYCODE"]== 1]
df_sevcode_2= df[df["SEVERITYCODE"]== 2]
df_sevcode_1_down= resample(df_sevcode_1,
                           replace= False,
                           n_samples= 55536,
                           random_state= 123)
df= pd.concat([df_sevcode_1_down, df_sevcode_2])
df

In [ ]:
df['SEVERITYCODE'].value_counts().to_frame()

Since most Machine Learning models require numerical data, we need to convert categorical variables to numerical ones. We will us one-hot enconding for this step.

In [ ]:
df2=pd.get_dummies(df[["ADDRTYPE"]])
df3=pd.get_dummies(df[["WEATHER"]])
df4=pd.get_dummies(df[["ROADCOND"]])
df5=pd.get_dummies(df[["LIGHTCOND"]])
df=pd.concat([df,df2,df3,df4,df5],axis=1)
df.drop(columns=["ADDRTYPE", "WEATHER", "ROADCOND", "LIGHTCOND"], inplace= True)

In [ ]:
df.head(20)

Now remove the columns with an 'Unknown' label since these do not provide addtional information

In [ ]:
df.dtypes

The data will be now standardised (i.e. every column re-scaled to have ~zero mean and ~unit variance) using the scikit learn StandardScaler routine. But first let's define the feature set(s).

In [ ]:
df['SEVERITYCODE']= df['SEVERITYCODE'].astype(int)

In [ ]:
X = df[['VEHCOUNT', 'UNDERINFL','HITPARKEDCAR','ADDRTYPE_Alley','ADDRTYPE_Block', 'ADDRTYPE_Intersection','WEATHER_Clear',
            'WEATHER_Elements','WEATHER_Overcast', 'ROADCOND_Dry','ROADCOND_Elements','LIGHTCOND_Daylight','LIGHTCOND_Night',
            'LIGHTCOND_Dusk/Dawn']].values.astype(float)

In [ ]:
from sklearn import preprocessing
X1 = preprocessing.StandardScaler().fit(X1).transform(X1.astype(float))
X1[0:5]

In [ ]:
y = df['SEVERITYCODE'].values
y[0:5]

### Model development

Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

#### 1- k-Nearest Neighbours (kNN) model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    KNNyhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, KNNyhat)

    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

In [ ]:
k = 8 
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

In [ ]:
KNNyhat = neigh.predict(X_test)

In [ ]:
print(KNNyhat [0:5])
print(y_test [0:5])

In [ ]:
print("Test set Accuracy: ", metrics.accuracy_score(y_test, KNNyhat))

#### 2- Decision Tree model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
severityTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
severityTree

In [ ]:
severityTree.fit(X_train,y_train)

In [ ]:
DTyhat = severityTree.predict(X_test)

In [ ]:
print (DTyhat [0:5])
print (y_test [0:5])

In [ ]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, DTyhat))

### 3- Support Vector Machine (SVM) model: 

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 

In [ ]:
SVMyhat = clf.predict(X_test)

In [ ]:
print (SVMyhat [0:5])
print (y_test [0:5])

In [ ]:
print("SVM Accuracy: ", metrics.accuracy_score(y_test, SVMyhat))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

def plot_coefficients(classifier, feature_names, top_features=20):
    coef = svm.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])

# create plot
plt.figure(figsize=(15, 5))
colors = [‘red’ if c < 0 else ‘blue’ for c in coef[top_coefficients]]
plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
feature_names = np.array(feature_names)
plt.xticks(np.arange(1, 1 + 2 * top_features), feature_names[top_coefficients], rotation=60, ha=’right’)
plt.show()cv = CountVectorizer()
cv.fit(data)
print len(cv.vocabulary_)
print cv.get_feature_names()X_train = cv.transform(data)

svm = LinearSVC()
svm.fit(X_train, target)plot_coefficients(svm, cv.get_feature_names())

## Model evaluation

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_similarity_score
import itertools

print('Jaccard Similarity Score:')
print('')
print('KNN model:', jaccard_similarity_score(y_test, KNNyhat))
print('Decision Tree model:', jaccard_similarity_score(y_test, DTyhat))
print('SVM model:', jaccard_similarity_score(y_test, SVMyhat))
print('-------------------')
print('F1 Score')
print('')
print('KNN model:', f1_score(y_test, KNNyhat, average='weighted'))
print('Decision Tree model:', f1_score(y_test, DTyhat, average='weighted'))
print('SVM model:', f1_score(y_test, SVMyhat, average='weighted'))
print('-------------------')

# **Results and Discussion**

In order to develop a model for predicting accident severity, the re-sampled, cleaned dataset was split in to testing and training sub-samples (containing 30% and 70% of the samples, respectively) using the scikit learn “train_test_split” method. In total, 3 models were trained and evaluated.

### KNN model 

The value of'k' was established by running kNN models for k=1–10 using the kNeighborsClassifier function from scikit learn. The model is optimised at k=8, at which the model correctly predicts accident severity 61% of the time. The Jaccard Index and F1 score are respectively 0.614 and 0.612.

### Decision Tree model 

A decision tree model was trained on the data according to the “entropy” criterion, and allowed to run until covergence. The decision tree correctly predicts accident severity 63% of the time and has Jaccard Index and F1 scores of 0.627 and 0.626 respectively.

### SVM model 

An SVM model was built using the scikit learn C-Support Vector Classification method (svm.svc), with a linear mapping kernel employed in order that the model could return a list of the features with the most diagnostic power for determining accident severity. The SVM model correctly predicts accident severity 63% of the time, and has Jaccard Index and F1 scores of 0.628 and 0.627 respectively.

From the model evaluation indexes, we can conclude that the 3 models have a similar capacity to predict accident severity.
Models could be further explored by changing the features set and see if prediction accuracy increases by removing or including features.

# **Conclusion**

Car accident data for the city of Seattle between 2004–2019 have been used to train and evaluate machine learning models for predicting accident severity based on the context of the accident. Three classes of models have been trained and evaluated: (i) k-Nearest Neighbors, (ii) Decision Tree and (iii) Support Vector Machine. The three models performed similarly, predicting correctly 62-63% of severity scores, with a slightly better performance by the SVM model. This work highlights that machine learning techniques can be used to probe historical data in order to make reliable predictions about the outcome of road traffic accidents, given information which is available at the time when an accident is reported. This model can be extended to include new features and can give city planners insight into the road conditions/features which are associated with higher accident severity. By predicting accident severity as a function of weather, date, location and road conditions, this model may be able to help aid the decision making of city roads planning.
